uninstall pytorch libraries 1.10 


# CARATE-BERT

In [ ]:
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 97.5 MB/s eta 0:00:13tcmalloc: large alloc 1147494400 bytes == 0x56234830c000 @  0x7f1aa0888615 0x56230f8153bc 0x56230f8f618a 0x56230f8181cd 0x56230f90ab3d 0x56230f88c458 0x56230f88702f 0x56230f819aba 0x56230f88c2c0 0x56230f88702f 0x56230f819aba 0x56230f888cd4 0x56230f90b986 0x56230f888350 0x56230f90b986 0x56230f888350 0x56230f90b986 0x56230f888350 0x56230f819f19 0x56230f85da79 0x56230f818b32 0x56230f88c1dd 0x56230f88702f 0x56230f819aba 0x56230f888cd4 0x56230f88702f 0x56230f819aba 0x56230f887eae 0x56230f8199da 0x56230f888108 0x56230f88702f
     |████████████████▌               | 1055.7 MB 1.2 MB/s eta 0:13:34tcmalloc: large alloc 1434370048 bytes == 0x56238c962000 @  0x7f1aa0888615 0x56230f8153bc 0x56230f8f618a 0x56230f8181cd 0x56230f90ab3d 0x56230f88c458 0x56230f88702f 0x56230f819aba 0x56230f88c2c0 0x56230f88702f 0x56230f819aba 0x56230f888cd4 0x56230f90b986 0x56230f888

# Install PytorchGeometric

In [1]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

     |████████████████████████████████| 7.9 MB 4.1 MB/s 
     |████████████████████████████████| 3.5 MB 3.8 MB/s 
     |████████████████████████████████| 145 kB 4.0 MB/s 
     |████████████████████████████████| 74 kB 4.3 MB/s 
     |████████████████████████████████| 112 kB 35.2 MB/s 
     |████████████████████████████████| 596 kB 55.5 MB/s 


In [2]:
!python -c "import torch; print(torch.version.cuda)"

11.1


In [3]:
import torch
print(torch.__version__)

1.10.0+cu111


In [4]:
!pip install rdkit-pypi

     |████████████████████████████████| 20.6 MB 105.2 MB/s 


In [2]:
%matplotlib inline
import torch
import networkx as nx
import matplotlib.pyplot as plt


def visualize(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])

    if torch.is_tensor(h):
        h = h.detach().cpu().numpy()
        plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
        if epoch is not None and loss is not None:
            plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    else:
        nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                         node_color=color, cmap="Set2")
    plt.show()

ModuleNotFoundError: No module named 'matplotlib'

#MCF-7

Breast Cancer actives to MCF-7 cells



In [6]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
import rdkit as rdkit


def load_data(dataset):
  path = '.'
  dataset = TUDataset(path, name=dataset).shuffle()

  train_dataset = dataset[len(dataset) // 20:]
  test_dataset = dataset[:len(dataset) // 20]

  print(f'Train Dataset: {train_dataset}:')
  print('======================')
  print(f'Number of graphs: {len(train_dataset)}')
  print(f'Number of features: {train_dataset.num_features}')
  print(f'Number of classes: {train_dataset.num_classes}')

  print(f'Test Dataset: {test_dataset}:')
  print('======================')
  print(f'Number of graphs: {len(test_dataset)}')
  print(f'Number of features: {test_dataset.num_features}')
  print(f'Number of classes: {test_dataset.num_classes}')



  test_loader = DataLoader(test_dataset, batch_size=64)
  train_loader = DataLoader(train_dataset, batch_size=64)
  return test_loader, train_loader, dataset, train_dataset, test_dataset 


In [7]:
print(torch_geometric.__version__)

NameError: ignored

In [8]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
import sklearn.metrics as metrics
from torch_geometric.nn import global_add_pool, GraphConv, GATConv
import json 

class Net(torch.nn.Module):
    def __init__(self, dim, dataset):
        super(Net, self).__init__()
        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return torch.sigmoid(x)

def train(epoch, model, device, train_loader, optimizer, num_classes = 2):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    correct = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = F.one_hot(data.y, num_classes = num_classes).type(torch.FloatTensor)
        #assert len(data.y) == len(train_loader.dataset), (str(len(train_loader.dataset))+" "+str(len(data.y)))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.5).float()
        loss = torch.nn.BCELoss()
        loss = loss(output_probs, data.y)
        loss.backward()
        optimizer.step()
        correct += (output == data.y).float().sum()/num_classes
    return correct / len(train_loader.dataset)


def test(loader, epoch, model, device, test=False):
    model.eval()

    correct = 0
    if test: 
      outs = []
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        #data.y = F.one_hot(data.y, num_classes = 6).type(torch.FloatTensor)
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        output = (output_probs > 0.5).float()
        correct += (torch.argmax(output, dim=1) == data.y).float().sum()
        #print(output, data.y)
        if test: 
          outs.append(output.cpu().detach().numpy())
    if test: 
      outputs =np.concatenate(outs, axis=0 ).astype(float)
      np.savetxt("MCF-7_epoch"+str(epoch)+".csv", outputs)
    return correct / len(loader.dataset)

def cv(data_set, n=5, num_epoch=150, num_classes = 2):
  result = []
  acc_store = []
  auc_store = []
  loss_store = [] 
  tmp = {}
  for i in range(n):
    test_loader, train_loader, dataset, train_dataset, test_dataset  = load_data(dataset=data_set)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net(dim=364, dataset=dataset).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

    for epoch in range(1, num_epoch):
        train_loss = train(epoch=epoch, model=model, device=device, optimizer=optimizer, train_loader=train_loader, num_classes = num_classes)
        loss_store.append(train_loss.cpu().tolist())
        train_acc = test(train_loader, device=device, model=model, epoch=epoch)
        test_acc = test(test_loader, device=device, model=model, epoch=epoch, test=True)
        acc_store.append([train_acc.cpu().tolist(), test_acc.cpu().tolist()])
        print('Epoch: {:03d}, Train Loss: {:.7f}, '
              'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                           train_acc, test_acc))
        y = np.zeros((len(test_dataset)))
        x = np.loadtxt("MCF-7_epoch"+str(epoch)+".csv")
        for i in range(len(test_dataset)):
          y[i] = test_dataset[i].y
        y = torch.as_tensor(y)
        y = F.one_hot(y.long(), num_classes = num_classes).long()
        store_auc = []
        for i in range(len(x[0,:])): 
          auc = metrics.roc_auc_score(y[:,i], x[:,i])
          print("AUC of "+str(i) +"is:", auc)
          store_auc.append(auc)
        auc_store.append(store_auc)
        
        if auc >=0.9:
          break
        tmp["Loss"] = list(loss_store)
        tmp["Acc"] = list(acc_store)
        tmp["AUC"] = auc_store
    with open("/content/drive/MyDrive/CARATE_RESULTS/"+data_set+"_"+str(i)+".csv", 'w') as f:
        json.dump(tmp, f)
        print("Saved iteration one to "+"/content/drive/MyDrive/CARATE_RESULTS/"+data_set+"_"+str(i)+".csv")
    result.append(tmp)          
  return result 


In [9]:
dataset = "MCF-7"
result = cv(data_set=dataset, n=5, num_epoch=300)

Extracting ./MCF-7/MCF-7.zip
Processing...
Done!
/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Train Dataset: MCF-7(26382):
Number of graphs: 26382
Number of features: 46
Number of classes: 2
Test Dataset: MCF-7(1388):
Number of graphs: 1388
Number of features: 46
Number of classes: 2
Epoch: 001, Train Loss: 0.9172542, Train Acc: 0.9176332, Test Acc: 0.9128242
AUC of 0is: 0.5
AUC of 1is: 0.5
Epoch: 002, Train Loss: 0.9176711, Train Acc: 0.9176332, Test Acc: 0.9128242
AUC of 0is: 0.5
AUC of 1is: 0.5
Epoch: 003, Train Loss: 0.9176711, Train Acc: 0.9178228, Test Acc: 0.9128242
AUC of 0is: 0.5
AUC of 1is: 0.5
Epoch: 004, Train Loss: 0.9172921, Train Acc: 0.9178607, Test Acc: 0.9149855
AUC of 0is: 0.5198718910421573
AUC of 1is: 0.5198718910421573
Epoch: 005, Train Loss: 0.9179365, Train Acc: 0.9183913, Test Acc: 0.9149855
AUC of 0is: 0.5240041224471159
AUC of 1is: 0.5198718910421573
Epoch: 006, Train Loss: 0.9184103, Train Acc: 0.9186566, Test Acc: 0.9157060
AUC of 0is: 0.5206611570247934
AUC of 1is: 0.5165289256198347
Epoch: 007, Train Loss: 0.9186377, Train Acc: 0.9179365, Test Acc

FileNotFoundError: ignored

In [1]:
dataset = "MOLT-4"
result = cv(data_set=dataset, n=5, num_epoch=300)


NameError: name 'cv' is not defined

In [ ]:
dataset = "Yeast"

result = cv(data_set=dataset, n=5, num_epoch=300)


In [ ]:
dataset = "PROTEINS"

result = cv(data_set=dataset, n=5, num_epoch=300)


In [ ]:
dataset = "ENZYMES"
result = cv(data_set=dataset, n=5, num_epoch=1000, num_classes=6)


# Regression
* investigate single and multiclass regression

In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
import rdkit as rdkit

def load_data(dataset, split = 20 ):
  path = '.'
  dataset = TUDataset(path, name=dataset).shuffle()

  train_dataset = dataset[len(dataset) // split:]
  test_dataset = dataset[:len(dataset) // split]

  print(f'Train Dataset: {train_dataset}:')
  print('======================')
  print(f'Number of graphs: {len(train_dataset)}')
  print(f'Number of features: {train_dataset.num_features}')
  print(f'Number of classes: {train_dataset.num_classes}')

  print(f'Test Dataset: {test_dataset}:')
  print('======================')
  print(f'Number of graphs: {len(test_dataset)}')
  print(f'Number of features: {test_dataset.num_features}')
  print(f'Number of classes: {test_dataset.num_classes}')



  test_loader = DataLoader(test_dataset, batch_size=64)
  train_loader = DataLoader(train_dataset, batch_size=64)

  return test_loader, train_loader, dataset, train_dataset, test_dataset 


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
import sklearn.metrics as metrics
from torch_geometric.nn import global_add_pool, GraphConv, GATConv

class Net(torch.nn.Module):
    def __init__(self, dim, dataset):
        super(Net, self).__init__()
        num_features = dataset.num_features
        self.dim = dim

        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return torch.sigmoid(x)

def train(epoch, model, factor, device, train_loader, optimizer, split = 20, num_classes = 2):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    mae = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = data.y/factor[0]
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch).flatten()
        loss = torch.nn.MSELoss()
        loss = loss(output_probs, data.y)
        loss_mae = torch.nn.L1Loss()
        mae += loss_mae(output_probs, data.y).item()
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
    return mae/len(train_loader)


def test(loader, epoch, model, device, test=False):
    model.eval()

    mae = 0
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = data.y/factor[0]
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        loss_mae = torch.nn.L1Loss()
        mae += loss_mae(output_probs, data.y).item()
        torch.cuda.empty_cache()
    return mae/len(loader)

def normalization_factor(data_set, num_classes): 
  import numpy as np
  import torch.nn.functional as F
  y = np.zeros((len(data_set),1, num_classes))
  for i in range(len(data_set)):
    y[i, :, :] = data_set[i].y
  factor = np.zeros((num_classes))
  for i in range(num_classes): 
    norm = np.linalg.norm(y[:,0,i], ord=2)
    factor[i] = norm
  return factor

def cv_regression(data_set, n=5, num_epoch=250, num_classes = 2):
  result = {}
  test_mae = []
  test_mse = []
  train_mae = [] 
  tmp = {}
  
  for i in range(n):
    test_loader, train_loader, dataset, train_dataset, test_dataset  = load_data(dataset=data_set)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net(dim=364, dataset=dataset).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
    factor = normalization_factor(data_set = dataset, num_classes=num_classes)
    for epoch in range(1, num_epoch):
        loss = train(epoch=epoch, model=model, factor = factor, device=device, optimizer=optimizer, train_loader=train_loader, num_classes = num_classes)
        train_mae.append(train_loss.cpu().tolist())
        #train_acc = test(train_loader, device=device, model=model, epoch=epoch)
        ts_mse, ts_mae = test(test_loader, device=device, model=model, epoch=epoch, test=True)
        test_mse.append(ts_mse.cpu().tolist())
        test_mae.append(ts_mase.cupu().tolist())
        print('Epoch: {:03d}, Train Loss: {:.7f}, '
              'Train Acc: {:.7f}, Test Acc: {:.7f}'.format(epoch, train_loss,
                                                           train_acc, test_acc))
        y = np.zeros((len(test_dataset)))
        x = np.loadtxt("MCF-7_epoch"+str(epoch)+".csv")
        for i in range(len(test_dataset)):
          y[i] = test_dataset[i].y
        y = torch.as_tensor(y)
        y = F.one_hot(y.long(), num_classes = num_classes).long()
        store_auc = []
        for i in range(len(x[0,:])): 
          auc = metrics.roc_auc_score(y[:,i], x[:,i])
          print("AUC of "+str(i) +"is:", auc)
          store_auc.append(auc)
        auc_store.append(store_auc)
        
        if auc >=0.9:
          break
        tmp["MAE Train"] = list(loss_store)
        tmp["MAE Test"] = list(acc_store)
        tmp["MSE Test"] = auc_store
    result[str(i)] = tmp 
          
  return result 


In [ ]:
dataset = "ZINC_full"
result = cv_regression(data_set=dataset, n=5, num_epoch=250, num_classes=1)
import csv
with open("/content/drive/MyDrive/CARATE_RESULTS/"+dataset+"2_20split.csv", 'w') as f:
    w = csv.writer(f)
    for k, v in result.items():
        w.writerow([k, v])

NameError: ignored

In [ ]:
dataset = "alchemy_full"
result = cv_regression(data_set=dataset, n=5, num_epoch=250, num_classes=1)
import csv
with open("/content/drive/MyDrive/CARATE_RESULTS/"+dataset+"_20split.csv", 'w') as f:
    w = csv.writer(f)
    for k, v in result.items():
        w.writerow([k, v])

Extracting ./alchemy_full/alchemy_full.zip
Processing...
Done!


Train Dataset: alchemy_full(192451):
Number of graphs: 192451
Number of features: 6
Number of classes: 12
Test Dataset: alchemy_full(10128):
Number of graphs: 10128
Number of features: 6
Number of classes: 12


/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


ValueError: ignored

## ZINC

In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
import rdkit as rdkit


def load_data(data_set):
  path = '.'
  dataset = TUDataset(path, name=data_set).shuffle()

  train_dataset = dataset[len(dataset) // 20:]
  test_dataset = dataset[:len(dataset) // 20]

  print(f'Train Dataset: {train_dataset}:')
  print('======================')
  print(f'Number of graphs: {len(train_dataset)}')
  print(f'Number of features: {train_dataset.num_features}')
  print(f'Number of classes: {train_dataset.num_classes}')

  print(f'Test Dataset: {test_dataset}:')
  print('======================')
  print(f'Number of graphs: {len(test_dataset)}')
  print(f'Number of features: {test_dataset.num_features}')
  print(f'Number of classes: {test_dataset.num_classes}')



  test_loader = DataLoader(test_dataset, batch_size=64)
  train_loader = DataLoader(train_dataset, batch_size=64)
  return test_loader, train_loader, dataset


In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv, BatchNorm

class Net(torch.nn.Module):
    def __init__(self, dim, dataset):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim
        #self.norm1 = BatchNorm(num_features)
        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, 1)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return x

def train(model, epoch, train_loader, optimizer, device, factor, num_classes):

    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    mae = 0
    mse = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = data.y/factor[0]
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch).flatten()
        loss = torch.nn.MSELoss()
        loss = loss(torch.reshape(output_probs, (-1,)), torch.reshape(data.y, (-1,)))
        #mse += loss.item()
        loss_mae = torch.nn.L1Loss()
        mae += loss_mae(torch.reshape(output_probs, (-1,)), torch.reshape(data.y, (-1,))).item()
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
    return mae/len(train_loader)

def test(model, loader, device, factor, num_classes):
    model.eval()

    mae = 0
    mse = 0
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = data.y/factor[0]
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        loss_mae = torch.nn.L1Loss()
        mae += loss_mae(torch.reshape(output_probs, (-1,)),torch.reshape(data.y, (-1,))).item()
        loss = torch.nn.MSELoss()
        mse += loss(torch.reshape(output_probs, (-1,)), torch.reshape(data.y, (-1,))).item()
        torch.cuda.empty_cache()
    return mae/len(loader), mse/len(loader)

def normalization_factor(data_set, num_classes): 
  import numpy as np
  import torch.nn.functional as F
  y = np.zeros((len(data_set),1, num_classes))
  for i in range(len(data_set)):
    y[i, :, :] = data_set[i].y
  factor = np.zeros((num_classes))
  for i in range(num_classes): 
    norm = np.linalg.norm(y[:,0,i], ord=2)
    factor[i] = norm
  return factor

def save_result(dataset, result): 
  import csv
  with open("/content/drive/MyDrive/CARATE_RESULTS/"+dataset+"_20split.csv", 'w') as f:
      w = csv.writer(f)
      for k, v in result.items():
          w.writerow([k, v])
def cv_regression(data_set, n = 5, num_epoch=150, num_classes=1): 
  result = {}
  test_mae = []
  test_mse = []
  train_mae = [] 
  train_mse = [] 
  tmp = {}
  
  for i in range(n):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_loader, test_loader, dataset = load_data(data_set=data_set)
    factor = normalization_factor(data_set=dataset, num_classes=1)
    model = Net(dim=364, dataset=dataset).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.000005)
    for epoch in range(1, num_epoch):
        mae = train(model=model, epoch=epoch, optimizer=optimizer, train_loader=train_loader, device=device, factor=factor, num_classes = num_classes)
        train_mae_val, train_mse_val = test(model, loader=train_loader, device=device, factor=factor, num_classes = num_classes)
        test_mae_val, test_mse_val = test(model, loader=test_loader, device=device, factor=factor, num_classes = num_classes)
        train_mae.append(train_mae_val)
        train_mse.append(train_mse_val)
        test_mse.append(test_mae_val)
        test_mse.append(test_mse_val)

        print('Epoch: {:03d}, Train MAE, MSE at epoch: ({:.7f}, {:.7f}), Test MAE, MSE at epoch: ({:.7f}, {:.7f})'.format(epoch, train_mae_val,
                                                      train_mse_val, test_mae_val, test_mse_val))
        torch.cuda.empty_cache()

    tmp["MAE Train"] = list(train_mae)
    tmp["MSE Train"] = list(train_mse)
    tmp["MAE Test"] = list(train_mae)
    tmp["MSE Test"] = list(train_mse)
    result[str(i)] = tmp 
    save_result(dataset=data_set, result=result)
  return result

In [ ]:
dataset = "ZINC_full"
result = cv_regression(data_set=dataset, n=1, num_epoch=250, num_classes=1)
import csv
with open("/content/drive/MyDrive/CARATE_RESULTS/"+dataset+"5_20split.csv", 'w') as f:
    w = csv.writer(f)
    for k, v in result.items():
        w.writerow([k, v])

Extracting ./ZINC_full/ZINC_full.zip
Processing...
Done!


Train Dataset: ZINC_full(236984):
Number of graphs: 236984
Number of features: 28
Number of classes: 247313
Test Dataset: ZINC_full(12472):
Number of graphs: 12472
Number of features: 28
Number of classes: 247313


/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 001, Train MAE, MSE at epoch: (0.0297123, 0.0012545), Test MAE, MSE at epoch: (0.0297109, 0.0012496)
Epoch: 002, Train MAE, MSE at epoch: (0.0279458, 0.0010718), Test MAE, MSE at epoch: (0.0279541, 0.0010793)


KeyboardInterrupt: ignored

In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
import rdkit as rdkit


def load_data(data_set):
  path = '.'
  dataset = TUDataset(path, name=data_set).shuffle()

  train_dataset = dataset[len(dataset) // 10:]
  test_dataset = dataset[:len(dataset) // 10]

  print(f'Train Dataset: {train_dataset}:')
  print('======================')
  print(f'Number of graphs: {len(train_dataset)}')
  print(f'Number of features: {train_dataset.num_features}')
  print(f'Number of classes: {train_dataset.num_classes}')

  print(f'Test Dataset: {test_dataset}:')
  print('======================')
  print(f'Number of graphs: {len(test_dataset)}')
  print(f'Number of features: {test_dataset.num_features}')
  print(f'Number of classes: {test_dataset.num_classes}')



  test_loader = DataLoader(test_dataset, batch_size=64)
  train_loader = DataLoader(train_dataset, batch_size=64)
  return test_loader, train_loader, dataset


In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv, BatchNorm

class Net(torch.nn.Module):
    def __init__(self, dim, dataset):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim
        #self.norm1 = BatchNorm(num_features)
        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return x
        

def train(model, epoch, train_loader, optimizer, device, std, mean, num_classes):

    model.train()

    if epoch % 51 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    mae = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        #data.y = (data.y)/factor
        data.y = (data.y.numpy()-mean)/std
        data.y = torch.from_numpy(data.y).type(torch.FloatTensor)
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch)
        loss = torch.nn.MSELoss()
       # print(data.y.shape)
        loss = loss(output_probs, data.y)
        #loss_mae = torch.nn.L1Loss()
        #mae += loss_mae(output_probs, data.y).item()
        mse = loss.item()
        loss.backward()
        optimizer.step()
        #error += torch.linalg.norm(output_probs - data.y, ord = 1)/32
        torch.cuda.empty_cache()
    return mse/len(train_loader)

def test(model, loader, device, std, mean, num_classes):
    model.eval()

    mae = 0
    mse = 0
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = (data.y.numpy()-mean)/std
        data.y = torch.from_numpy(data.y).type(torch.FloatTensor)

        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        #error += torch.linalg.norm(output_probs - data.y, ord = 1)/32
        loss = torch.nn.MSELoss()
        loss = loss(output_probs, data.y)
        mse += loss.item()
        loss_mae = torch.nn.L1Loss()
        mae += loss_mae(output_probs, data.y).item()
        torch.cuda.empty_cache()
    return (mae/len(loader), mse/len(loader))

def normalization_factor(data_set, num_classes): 
  import numpy as np
  import torch.nn.functional as F
  y = np.zeros((len(data_set),1, num_classes))
  for i in range(len(data_set)):
    y[i, :] = data_set[i].y
  factor = np.zeros((1,1,num_classes))
  mean = np.zeros((1,1,num_classes))
  std = np.zeros((1,1,num_classes))
  for i in range(num_classes): 
    norm = np.linalg.norm(y[:,0,i], ord=2)
    factor[:,:,i] =  norm
    mean[:,:,i] =   np.mean(y[:,0,i])
    std[:,:,i] = np.std(y[:,0,i])
    return factor, mean, std

def save_result(dataset, result): 
  import csv
  with open("/content/drive/MyDrive/CARATE_RESULTS/"+dataset+"_20split.csv", 'w') as f:
      w = csv.writer(f)
      for k, v in result.items():
          w.writerow([k, v])
def cv_regression(data_set, n = 5, num_epoch=150, num_classes=1): 
  result = {}
  test_mae = []
  test_mse = []
  train_mae = [] 
  train_mse = [] 
  tmp = {}
  
  for i in range(n):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_loader, test_loader, dataset = load_data(data_set=data_set)
    factor, mean, std = normalization_factor(data_set=dataset, num_classes=num_classes)
    model = Net(dim=364, dataset=dataset).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.000005)
    for epoch in range(1, num_epoch):
        mae = train(model=model, epoch=epoch, optimizer=optimizer, train_loader=train_loader, device=device, mean= mean, std=std, num_classes = num_classes)
        train_mae_val, train_mse_val = test(model, loader=train_loader, device=device, mean= mean, std=std, num_classes = num_classes)
        test_mae_val, test_mse_val = test(model, loader=test_loader, device=device, mean= mean, std=std, num_classes = num_classes)
        train_mae.append(train_mae_val)
        train_mse.append(train_mse_val)
        test_mse.append(test_mae_val)
        test_mse.append(test_mse_val)

        print('Epoch: {:03d}, Train MAE, MSE at epoch: ({:.7f}, {:.7f}), Test MAE, MSE at epoch: ({:.7f}, {:.7f})'.format(epoch, train_mae_val,
                                                      train_mse_val, test_mae_val, test_mse_val))
        torch.cuda.empty_cache()

    tmp["MAE Train"] = list(train_mae)
    tmp["MSE Train"] = list(train_mse)
    tmp["MAE Test"] = list(train_mae)
    tmp["MSE Test"] = list(train_mse)
    result[str(i)] = tmp 
    save_result(dataset=data_set, result=result)
  return result

In [ ]:
data_name = "alchemy_full"
result = cv_regression(data_set=data_name, n=1, num_epoch=250, num_classes=12)
import csv
with open("/content/drive/MyDrive/CARATE_RESULTS/"+data_name+"1_10split.csv", 'w') as f:
    w = csv.writer(f)
    for k, v in result.items():
        w.writerow([k, v])

Extracting ./alchemy_full/alchemy_full.zip
Processing...
Done!


Train Dataset: alchemy_full(182322):
Number of graphs: 182322
Number of features: 6
Number of classes: 12
Test Dataset: alchemy_full(20257):
Number of graphs: 20257
Number of features: 6
Number of classes: 12


/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1, 64, 12])) that is different to the input size (torch.Size([64, 12])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([1, 33, 12])) that is different to the input size (torch.Size([33, 12])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 001, Train MAE, MSE at epoch: (nan, nan), Test MAE, MSE at epoch: (nan, nan)
Epoch: 002, Train MAE, MSE at epoch: (nan, nan), Test MAE, MSE at epoch: (nan, nan)
Epoch: 003, Train MAE, MSE at epoch: (nan, nan), Test MAE, MSE at epoch: (nan, nan)
Epoch: 004, Train MAE, MSE at epoch: (nan, nan), Test MAE, MSE at epoch: (nan, nan)
Epoch: 005, Train MAE, MSE at epoch: (nan, nan), Test MAE, MSE at epoch: (nan, nan)
Epoch: 006, Train MAE, MSE at epoch: (nan, nan), Test MAE, MSE at epoch: (nan, nan)
Epoch: 007, Train MAE, MSE at epoch: (nan, nan), Test MAE, MSE at epoch: (nan, nan)
Epoch: 008, Train MAE, MSE at epoch: (nan, nan), Test MAE, MSE at epoch: (nan, nan)
Epoch: 009, Train MAE, MSE at epoch: (nan, nan), Test MAE, MSE at epoch: (nan, nan)
Epoch: 010, Train MAE, MSE at epoch: (nan, nan), Test MAE, MSE at epoch: (nan, nan)
Epoch: 011, Train MAE, MSE at epoch: (nan, nan), Test MAE, MSE at epoch: (nan, nan)
Epoch: 012, Train MAE, MSE at epoch: (nan, nan), Test MAE, MSE at epoch: (na

In [ ]:
# Apparently, the algorithm converges to numerical accuracy

## Alchemy

In [ ]:
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
import torch
import rdkit as rdkit

path = '.'
dataset = TUDataset(path, name="alchemy_full").shuffle()
train_dataset = dataset[len(dataset) // 20:]
test_dataset = dataset[:len(dataset) // 20]

In [ ]:
print(f'Train Dataset: {train_dataset}:')
print('======================')
print(f'Number of graphs: {len(train_dataset)}')
print(f'Number of features: {train_dataset.num_features}')
print(f'Number of classes: {train_dataset.num_classes}')

print(f'Test Dataset: {test_dataset}:')
print('======================')
print(f'Number of graphs: {len(test_dataset)}')
print(f'Number of features: {test_dataset.num_features}')
print(f'Number of classes: {test_dataset.num_classes}')



test_loader = DataLoader(test_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64)


Train Dataset: alchemy_full(192451):
Number of graphs: 192451
Number of features: 6
Number of classes: 12
Test Dataset: alchemy_full(10128):
Number of graphs: 10128
Number of features: 6
Number of classes: 12


/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
import numpy as np
import torch.nn.functional as F
y = np.zeros((len(dataset),1, 12))
for i in range(len(dataset)):
  y[i, :, :] = dataset[i].y
factor = np.zeros((1,1,12))
add = np.zeros((1,1,12))
std = np.zeros((1,1,12))
for i in range(12): 
  norm = np.linalg.norm(y[:,0,i], ord=2)
  factor[:,:,i] =  norm
  add[:,:,i] =   np.mean(y[:,0,i])
  std[:,:,i] = np.std(y[:,0,i])
print(np.mean(add))
print(factor)
print(np.mean(std))
print(y/factor)
print(dataset[789].y/factor[0])

-15.716988681808667
[[[8.61489461e+01 2.83672304e-02 1.08626915e+02 2.28857479e+05
   1.03899945e+02 2.28836977e+05 4.21827207e+04 2.28841587e+05
   2.28836566e+05 2.03088601e+01 1.39834646e+03 7.75887194e+05]]]
72.92028404696889
[[[ 2.61137958e-03  2.72144998e-03  2.54694320e-03 ...  7.01687034e-04
    1.75037497e-03  3.18982244e-03]]

 [[ 3.22365996e-03  2.74260130e-03  3.05092980e-03 ...  4.14751281e-03
    9.45250282e-05  2.42749971e-03]]

 [[ 2.58331369e-03  2.35482979e-03  2.20499279e-03 ...  2.55638449e-03
    1.35500355e-03  2.30521719e-03]]

 ...

 [[ 2.07553955e-03  2.21382197e-03  1.81698601e-03 ... -1.14862677e-03
    1.92178836e-03  3.81268363e-03]]

 [[ 3.55083604e-03  3.08807016e-03  3.03673330e-03 ...  3.43613907e-03
    1.16309696e-03  3.38122946e-03]]

 [[ 1.92963871e-03  2.30900218e-03  1.76669059e-03 ... -1.39286233e-03
    1.91557720e-03  4.01155658e-03]]]
tensor([[ 0.0018,  0.0020,  0.0019, -0.0017, -0.0022, -0.0017,  0.0021, -0.0017,
         -0.0017, -0.0009,  0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import global_add_pool, GraphConv, GATConv, BatchNorm

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()

        num_features = dataset.num_features
        self.dim = dim
        #self.norm1 = BatchNorm(num_features)
        self.conv1 = GraphConv(num_features, dim)
        #self.conv2 = GraphConv(dim, dim)
        self.conv3 = GATConv(dim, dim, dropout = 0.6)
        self.conv5 = GraphConv(dim, dim)

        self.fc1 = Linear(dim, dim)
        self.fc2 = Linear(dim, dataset.num_classes)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        #x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        x = F.relu(self.conv5(x, edge_index, edge_weight))
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        return x

def train(epoch):

    model.train()

    if epoch % 51 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    mse = 0
    for data in train_loader:
        data.x = data.x.type(torch.FloatTensor)
        #data.y = (data.y)/factor
        data.y = (data.y.numpy())/factor
        data.y = torch.from_numpy(data.y).type(torch.FloatTensor)
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        optimizer.zero_grad()
        output_probs = model(data.x, data.edge_index, data.batch)
        loss = torch.nn.MSELoss()
       # print(data.y.shape)
        loss = loss(output_probs, data.y[0,:,:])
        #loss_mae = torch.nn.L1Loss()
        #mae += loss_mae(output_probs, data.y).item()
        mse += loss.item()
        loss.backward()
        optimizer.step()
        #error += torch.linalg.norm(output_probs - data.y, ord = 1)/32
        torch.cuda.empty_cache()
    return mse/len(train_loader)

def test(loader):
    model.eval()

    mae = 0
    mse = 0
    for data in loader:
        data.x = data.x.type(torch.FloatTensor)
        data.y = data.y.numpy()/factor[0]
        data.y = torch.from_numpy(data.y).type(torch.FloatTensor)
        data.y = torch.nan_to_num(data.y.type(torch.FloatTensor))
        data = data.to(device)
        output_probs = model(data.x, data.edge_index, data.batch)
        #error += torch.linalg.norm(output_probs - data.y, ord = 1)/32
        loss = torch.nn.MSELoss()
        loss = loss(output_probs, data.y)
        mse += loss.item()
        loss_mae = torch.nn.L1Loss()
        mae += loss_mae(output_probs, data.y).item()
        torch.cuda.empty_cache()
    return (mae/len(loader), mse/len(loader))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=364).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



tmp = {}



for i in range(5):
  result = {}
  test_mae = []
  test_mse = []
  train_mae = [] 
  train_mse = [] 
  for epoch in range(1, 50):
      train_loss = train(epoch)
      train_mae_val, train_mse_val = test(train_loader)
      test_mae_val, test_mse_val = test(test_loader)
      train_mae.append(train_mae_val)
      train_mae.append(train_mse_val)
      test_mae.append(test_mae_val)
      test_mse.append(test_mse_val)
      print('Epoch: {:03d}, Train Loss at epoch: {:.7f}, '
            'Train MAE, MSE at epoch: ({},{}), Test MAE,MSE at epoch: ({},{})'.format(epoch, train_loss,
                                                    train_mae_val, train_mse_val, test_mae_val, test_mse_val))
      torch.cuda.empty_cache()
  tmp["MAE Train"] = list(train_mae)
  tmp["MSE Train"] = list(train_mse)
  tmp["MAE Test"] = list(train_mae)
  tmp["MSE Test"] = list(train_mse)
  result[str(i)] = tmp 
  import csv
  with open("/content/drive/MyDrive/CARATE_RESULTS/"+"alchemy_full_"+str(i)+"_10split.csv", 'w') as f:
      w = csv.writer(f)
      for k, v in result.items():
          w.writerow([k, v])

Epoch: 001, Train Loss at epoch: 0.0014489, Train MAE, MSE at epoch: (0.0005177858969971644,7.273566194733254e-07), Test MAE,MSE at epoch: (0.000513349596774147,7.15892937729886e-07)
Epoch: 002, Train Loss at epoch: 0.0000007, Train MAE, MSE at epoch: (0.0005043527221556709,6.811765327021799e-07), Test MAE,MSE at epoch: (0.0004997818268405028,6.698086388427808e-07)
Epoch: 003, Train Loss at epoch: 0.0000007, Train MAE, MSE at epoch: (0.0005300315530822697,6.711402049238726e-07), Test MAE,MSE at epoch: (0.0005261032481871123,6.620683938359983e-07)
Epoch: 004, Train Loss at epoch: 0.0000007, Train MAE, MSE at epoch: (0.0006528387998224253,8.27719101221766e-07), Test MAE,MSE at epoch: (0.0006469085270029334,8.14835625433204e-07)
Epoch: 005, Train Loss at epoch: 0.0000299, Train MAE, MSE at epoch: (0.0005049738928720855,6.831383454853263e-07), Test MAE,MSE at epoch: (0.0005005324457452849,6.713595436002999e-07)
Epoch: 006, Train Loss at epoch: 0.0000007, Train MAE, MSE at epoch: (0.0005627